In [27]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [5]:
import transformers

print(transformers.__version__)

4.25.1


In [7]:
task = "sst2"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset

In [8]:
from datasets import load_dataset, load_metric

In [9]:
dataset = load_dataset("glue", task)
metric = load_metric('glue', task)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-9-28c988d01b75>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [11]:
dataset["train"][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [12]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(dataset["train"])

,sentence,label,idx
0,great rewards,positive,30267
1,"ode to unconditional love and compassion garnered from years of seeing it all , a condition only the old are privy to , and ... often misconstrued as weakness",positive,58800
2,old pickup skidding,negative,48110
3,cool .,positive,31792
4,a director i admire,positive,20150
5,into hokum,negative,35169
6,"wearing a cloak of unsentimental , straightforward text",positive,39152
7,nubile young actors,positive,40900
8,epics,positive,25653
9,enchanting ...,positive,10119


## Preprocessing the data

In [16]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
sentence1_key, sentence2_key = ("sentence", None)
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: hide new secretions from the parental units 


In [20]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [21]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], [101, 3397, 2053, 15966, 1010, 2069, 4450, 2098, 18201, 2015, 102], [101, 2008, 7459, 2049, 3494, 1998, 10639, 2015, 2242, 2738, 3376, 2055, 2529, 3267, 102], [101, 3464, 12580, 8510, 2000, 3961, 1996, 2168, 2802, 102], [101, 2006, 1996, 5409, 7195, 1011, 1997, 1011, 1996, 1011, 11265, 17811, 18856, 17322, 2015, 1996, 16587, 2071, 2852, 24225, 2039, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [22]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## Fine-tuning the model

In [23]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

In [24]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [39]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_precision = load_metric("precision")
   load_recall = load_metric("recall")
   load_f1 = load_metric("f1")
   load_accuracy = load_metric("accuracy")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   precision = load_precision.compute(predictions=predictions, references=labels)["precision"]
   recall = load_recall.compute(predictions=predictions, references=labels)["recall"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   return {"precision": precision, "recall": recall, "accuracy": accuracy, "f1": f1}


In [40]:
validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/distilbert-base-uncased-finetuned-sst2 is already a clone of https://huggingface.co/Ivor22/distilbert-base-uncased-finetuned-sst2. Make sure you pull the latest changes with `repo.git_pull()`.


In [41]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 67349
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 21050
  Number of trainable parameters = 66955010


Epoch,Training Loss,Validation Loss,Precision,Recall,Accuracy,F1
1,0.023700,0.663915,0.868476,0.936937,0.895642,0.901408
2,0.024700,0.577634,0.903803,0.909910,0.904817,0.906846
3,0.030400,0.653282,0.883871,0.925676,0.900229,0.904290
4,0.028100,0.665413,0.887689,0.925676,0.902523,0.906284
5,0.009500,0.783232,0.871036,0.927928,0.893349,0.898582


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-sst2/checkpoint-4210
Configuration saved in distilbert-base-uncased-finetuned-sst2/checkpoint-4210/config.json
Model weights saved in distilbert-base-uncased-finetuned-sst2/checkpoint-4210/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-sst2/checkpoint-4210/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-sst2/checkpoint-4210/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-sst2/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-sst

TrainOutput(global_step=21050, training_loss=0.021357785454838407, metrics={'train_runtime': 1610.2177, 'train_samples_per_second': 209.13, 'train_steps_per_second': 13.073, 'total_flos': 3066228568648968.0, 'train_loss': 0.021357785454838407, 'epoch': 5.0})

In [42]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16


{'eval_loss': 0.5776337385177612,
 'eval_precision': 0.9038031319910514,
 'eval_recall': 0.9099099099099099,
 'eval_accuracy': 0.9048165137614679,
 'eval_f1': 0.9068462401795736,
 'eval_runtime': 5.2014,
 'eval_samples_per_second': 167.648,
 'eval_steps_per_second': 10.574,
 'epoch': 5.0}

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

In [43]:
trainer.push_to_hub()

Saving model checkpoint to distilbert-base-uncased-finetuned-sst2
Configuration saved in distilbert-base-uncased-finetuned-sst2/config.json
Model weights saved in distilbert-base-uncased-finetuned-sst2/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-sst2/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-sst2/special_tokens_map.json
Several commits (4) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/255M [00:00<?, ?B/s]

Upload file runs/Jan15_18-21-34_9f6944aa26cd/events.out.tfevents.1673817053.9f6944aa26cd.951.10: 100%|########…

Upload file runs/Jan15_18-21-34_9f6944aa26cd/events.out.tfevents.1673815437.9f6944aa26cd.951.8:  26%|##5      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Ivor22/distilbert-base-uncased-finetuned-sst2
   2bde634..4aae9f7  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Ivor22/distilbert-base-uncased-finetuned-sst2
   2bde634..4aae9f7  main -> main

To https://huggingface.co/Ivor22/distilbert-base-uncased-finetuned-sst2
   4aae9f7..f98b81e  main -> main

   4aae9f7..f98b81e  main -> main



'https://huggingface.co/Ivor22/distilbert-base-uncased-finetuned-sst2/commit/4aae9f7af3ea745dca492b249821b8a5e56273b6'